In [1]:
import sys
from itertools import count
from torch import autograd
from torch_geometric.utils import dense_to_sparse
import copy
from collections import defaultdict

sys.path.append('../../')

from src.models.gcn import *
from src.utils.datasets import *
from src.models.trainable import *
from src.attacks.greedy_gd import *

print(sys.executable)

/home/niyati/miniconda3/envs/ersp_v2/bin/python


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [11]:
# dataset_directory = "../Cora"
cora_dataset = Planetoid(root='', name='Citeseer')
data = cora_dataset[0].to(device)
print(data)

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])


In [12]:
model = GCN(data.x.shape[1], cora_dataset.num_classes, [64]).to(device)

In [13]:
model.reset_parameters()
train = Trainable(model)
train.fit(data, 200)

Epoch 0, Train Loss - 2.764373540878296, Val Loss - 4.158348083496094, Val Accuracy - 0.351
Epoch 20, Train Loss - 0.1840253323316574, Val Loss - 3.8556249141693115, Val Accuracy - 0.628
Epoch 40, Train Loss - 0.008632137440145016, Val Loss - 4.5529398918151855, Val Accuracy - 0.608
Epoch 60, Train Loss - 0.12447350472211838, Val Loss - 4.521944046020508, Val Accuracy - 0.621
Epoch 80, Train Loss - 0.053464215248823166, Val Loss - 4.8211164474487305, Val Accuracy - 0.614
Epoch 100, Train Loss - 0.021316593512892723, Val Loss - 5.988728046417236, Val Accuracy - 0.569
Epoch 120, Train Loss - 0.008159114979207516, Val Loss - 6.669020175933838, Val Accuracy - 0.562
Epoch 140, Train Loss - 0.11458323150873184, Val Loss - 5.86370849609375, Val Accuracy - 0.616
Epoch 160, Train Loss - 0.003685181261971593, Val Loss - 5.529467582702637, Val Accuracy - 0.609
Epoch 180, Train Loss - 0.02688351646065712, Val Loss - 5.455043315887451, Val Accuracy - 0.594
Epoch 200, Train Loss - 0.0043840827420353

In [14]:
# Get initial accuracy
initial_loss, initial_accuracy = train.test(data)
print(f"Initial Accuracy: {initial_accuracy}")
print(f"Initial Loss: {initial_loss}")

Initial Accuracy: 0.608
Initial Loss: 5.282741069793701


In [15]:
attacker.num_budgets

5278

In [16]:
from collections import defaultdict 
from torch_geometric.utils import to_networkx

# Step 1: Initialize and run Metattack
attacker = Metattack(data, device=device)

attacker.setup_surrogate(model,
                         labeled_nodes=data.train_mask,
                         unlabeled_nodes=data.test_mask,
                         lambda_=0.)
attacker.reset_parameters(seed=42)
attacker.reset()

Metattack(
  (surrogate): GCN(
    (conv): Sequential(
      (0): GCNConv()
      (1): ReLU()
      (2): Dropout(p=0.5, inplace=False)
      (3): GCNConv()
    )
  )
)

In [17]:
#200% perturbation (Large Pool)
attacker.attack(1.0)

Peturbing graph...:   0%|          | 0/4552 [00:00<?, ?it/s]

Metattack(
  (surrogate): GCN(
    (conv): Sequential(
      (0): GCNConv()
      (1): ReLU()
      (2): Dropout(p=0.5, inplace=False)
      (3): GCNConv()
    )
  )
)

In [18]:
# Collect added and removed edges
added_edges = list(attacker._added_edges.keys())
removed_edges = list(attacker._removed_edges.keys())

# one list
all_perturbed_edges = added_edges + removed_edges


print(f"Added edges: {len(added_edges)}")
print(f"Removed edges: {len(removed_edges)}")
print(f"Total perturbed edges: {len(all_perturbed_edges)}")
print(f"Sample edges: {all_perturbed_edges[:5]}")

Added edges: 1358
Removed edges: 38
Total perturbed edges: 1396
Sample edges: [(43, 2566), (99, 2915), (113, 2788), (91, 631), (91, 3173)]


In [19]:
print("Unique perturbed edges (undirected):", len(set(frozenset(e) for e in all_perturbed_edges)))

Unique perturbed edges (undirected): 1375


In [20]:
attacker.num_budgets

4552

In [21]:
attacker.adj_changes.sum()

tensor(2654., device='cuda:0', grad_fn=<SumBackward0>)

In [22]:
from torch_geometric.utils import dense_to_sparse, to_networkx, from_networkx
import networkx as nx

In [23]:
G = to_networkx(data, to_undirected=True)
initial_edge_count = G.number_of_edges() // 2
ptb_rate = 0.20
budget = int(ptb_rate * initial_edge_count)

In [24]:
print(len(all_perturbed_edges))
print(budget)

1396
455


In [25]:
def two_phase_attack(split):
    diff_threshold = 0.01
    first_phase_edges = int(budget * split)
    second_phase_percent = ptb_rate * (1 - split) * 0.5
    print(f"\n--- Running split: {split} ---")
    print(f"Second phase perturbation rate: {second_phase_percent:.4f}")

    phase1_accuracies = []
    phase2_accuracies = []

    G = to_networkx(data, to_undirected=True)
    data_copy = copy.copy(data)

    i, j = 0, 0  # i - edges successfully added, j - index in list

    # === Phase 1 ===
    while i < first_phase_edges:
        if j >= len(all_perturbed_edges):
            print("Ran out of candidate edges in Phase 1. Moving to Phase 2.")
            break

        u, v = all_perturbed_edges[j]
        G.add_edge(u, v)

        modified_data = from_networkx(G).to(device)
        modified_data.x = data.x 
        modified_data.y = data.y 
        modified_data.train_mask = data.train_mask
        modified_data.test_mask = data.test_mask

        _, modified_accuracy = train.test(modified_data)

        if modified_accuracy == initial_accuracy:
            i += 1
            phase1_accuracies.append(modified_accuracy)
        else:
            G.remove_edge(u, v)

        j += 1

    print(f"Phase 1: Added {i} edges out of requested {first_phase_edges}.")

    # === Phase 2 ===
    modified_data = from_networkx(G).to(device)
    modified_data.x = data.x 
    modified_data.y = data.y 
    modified_data.train_mask = data.train_mask
    modified_data.test_mask = data.test_mask

    attacker = Metattack(modified_data, device=device)
    attacker.setup_surrogate(model,
                             labeled_nodes=data.train_mask,
                             unlabeled_nodes=data.test_mask, 
                             lambda_=0.)
    attacker.reset()
    attacker.attack(second_phase_percent)

    degs = defaultdict(tuple)
    for k, v in attacker._added_edges.items():
        degs[v] = (k, True)
    for k, v in attacker._removed_edges.items():
        degs[v] = (k, False)

    for _, second in degs.items():
        u, v = second[0]
        if second[1]:
            G.add_edge(u, v)
        else:
            G.remove_edge(u, v)

        modified_data = from_networkx(G).to(device)
        modified_data.x = data.x 
        modified_data.y = data.y 
        modified_data.train_mask = data.train_mask
        modified_data.test_mask = data.test_mask

        _, modified_accuracy = train.test(modified_data)
        phase2_accuracies.append(modified_accuracy)

    # === Final Reporting ===
    final_accuracy = phase2_accuracies[-1] if phase2_accuracies else (
        phase1_accuracies[-1] if phase1_accuracies else initial_accuracy)
    accuracy_drop = initial_accuracy - final_accuracy

    print(f"Final Accuracy: {final_accuracy:.4f}")
    print(f"Accuracy Drop: {accuracy_drop:.4f}")

    return {
        "split": split,
        "phase1_added": i,
        "phase1_accuracies": phase1_accuracies,
        "phase2_accuracies": phase2_accuracies,
        "final_accuracy": final_accuracy,
        "accuracy_drop": accuracy_drop
    }


In [26]:
splits = [0, 0.5]
split_dic = defaultdict(list)

In [27]:
for s in splits:
    print(s)
    split_dic[s] = two_phase_attack(s)

0

--- Running split: 0 ---
Second phase perturbation rate: 0.1000
Phase 1: Added 0 edges out of requested 0.


Peturbing graph...:   0%|          | 0/455 [00:00<?, ?it/s]

Final Accuracy: 0.5890
Accuracy Drop: 0.0190
0.5

--- Running split: 0.5 ---
Second phase perturbation rate: 0.0500
Phase 1: Added 227 edges out of requested 227.


Peturbing graph...:   0%|          | 0/238 [00:00<?, ?it/s]

Final Accuracy: 0.6050
Accuracy Drop: 0.0030


In [19]:
G = to_networkx(data, to_undirected=True)
initial_edge_count = G.number_of_edges() // 2
ptb_rate = 0.5
budget = int(ptb_rate * initial_edge_count)